__IMDB__

In [ ]:
from tqdm import tqdm
import requests  
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_page_IMDB(p):
    
    url = 'https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc&start={}&ref_=adv_nxt'.format(50*p+1)
    
    response = requests.get(url)
    
    tree = BeautifulSoup(response.content, 'html.parser')
    
    movies = tree.find_all('div', {'class': 'lister-item'})
    
    infaIMDB = [ ]
    d = {}

    for movie in movies:
        try:
            d = {'title': movie.h3.a.get_text(),
                 'year': int(''.join(filter(str.isdigit, movie.select('.lister-item-year')[0].get_text().strip()))),
                 'runtime': int(''.join(filter(str.isdigit, movie.select('.runtime')[0].get_text().strip()))),
                 'genre': movie.select('.genre')[0].get_text().strip().split(', '),
                 'imdb_rating': float(movie.strong.get_text()), 
                 'votes': int(''.join(filter(str.isdigit, 
                                             movie.select('.sort-num_votes-visible')[0].get_text().strip().split()[1])))}
            try:
                d['certificate'] = movie.select('.certificate')[0].get_text().strip()
            except:
                d['certificate'] = 'NaN'
            try: 
                d['metascore'] = int(movie.select('.ratings-metascore')[0].get_text().strip().split()[0])
            except:
                d['metascore'] = 0
            try: 
                d['gross'] = int(''.join(filter(str.isdigit, 
                                            movie.select('.sort-num_votes-visible')[0].select('span')[4].get('data-value'))))
            except:
                d['gross'] = 0
                
            infaIMDB.append(d)

        except:
            continue

    return infaIMDB

In [ ]:
infaIMDB = []

for p in tqdm(range(1,2)):
    infaIMDB.extend(get_page_IMDB(p))

In [ ]:
dfIMDB = pd.DataFrame(infaIMDB)
print(dfIMDB.shape)
dfIMDB.tail(15)

__Кинопоиск__

In [2]:
from fake_useragent import UserAgent

def get_page_KP(p):

    UserAgent().chrome
    
    url = 'https://www.kinopoisk.ru/top/navigator/m_act[rating]/1%3A/m_act[is_film]/on/order/num_vote/page/{}/#results'.format(p)
    
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    
    tree = BeautifulSoup(response.content, 'html.parser')
    
    movies = tree.find_all('div', {'class': 'item'})
    
    infaKP = [ ]
    
    for movie in movies:
        try:
            infaKP.append({'title': movies.find_all('div', {'class': 'name'})[0].select('a')[0].get_text(),
                           'year': movie.find_all('div', {'class': 'name'})[0].get_text().split('(')[1].split(')')[0],
                           ###сейчас в датаесете нет year, но мы его добавим в следующем этапе работы
                           'country': movies.find_all('span', {'class': 'gray_text'})[0].get_text().strip().replace('...', '').replace(',', '').split()[0],
                           'director': movie.find_all('a', {'class': "lined"})[0].get_text(),
                           'actor_1': movie.find_all('a', {'class': "lined"})[1].get_text(),
                           'actor_2': movie.find_all('a', {'class': "lined"})[2].get_text(),
                           'ratingKP': float(movie.find_all('div', {'class': "numVote"})[0].get_text().split()[0])})
        
        except:
            continue
        

    return infaKP

In [4]:
infaKP = []
d = {}

for p in tqdm(range(1,2)):
    infaKP.extend(get_page_KP(p))

 18%|▉    | 9/50 [00:08<00:39,  1.05it/s]


KeyboardInterrupt: 

In [ ]:
len(infaKP)

In [ ]:
dfKP = pd.DataFrame(infaKP)
print(dfKP.shape)
dfKP.tail(15)

__1000 звезд__

In [ ]:
for star in stars:
    try:
        stars_infa.append({'index': star.find_all('span', {'class': 'lister-item-index'})[0].get_text().split('.')[0],
                           'name': star.a.get_text().strip()}) 

In [ ]:
def get_page_stars(p):
    
    url = 'https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page=1'.format(p)
    
    response = requests.get(url)
    
    tree = BeautifulSoup(response.content, 'html.parser')
    
    stars = tree.find_all('div', {'class': 'lister-item-content'})
    
    stars_infa = [ ]

    for star in stars:
        try:
            stars_infa.append({'index': star.find_all('span', {'class': 'lister-item-index'})[0].get_text().split('.')[0],
                           'name': star.a.get_text().strip()}) 

        except:
            continue

    return stars_infa

In [ ]:
stars_infa = []
d = {}

for p in tqdm(range(1,2)):
    stars_infa.extend(get_page_stars(p))

In [ ]:
dfstars = pd.DataFrame(stars_infa)
print(dfstars.shape)
dfstars.head(10)

__фильмы с mail.ru__

In [ ]:
from fake_useragent import UserAgent

def get_page_mail(p):

    UserAgent().chrome
    
    url = 'https://kino.mail.ru/cinema/all/?order=rate_count&page={}'.format(p)
    
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    
    tree = BeautifulSoup(response.content, 'html.parser')
    
    movies = tree.find_all('div', {'class': 'p-itemevent-small__info'})
    
    mail_infa = [ ]
    
    for movie in movies:
        try:
            mail_infa.append({'title': movie.find_all('span', {'class': 'link__text'})[0].get_text(),
                           'country': movie.find_all('div', {'class': 'margin_top_5'})[0].get_text().split(',')[0]})

        except:
            continue
        

    return mail_infa

In [ ]:
mail_infa = []

for p in tqdm(range(1,2)):
    mail_infa.extend(get_page_mail(p))

In [ ]:
dfmail = pd.DataFrame(mail_infa)
print(dfmail.shape)
dfmail.head(10)